In [312]:
import pandas as pd
import numpy as np
import os
from glob import glob
import re
import io
import csv

In [377]:
# list paths of data in .mpt file
datafilespath = glob('data/*.mpt') 

In [378]:
datafilespath

['data\\20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_16_GCPL6_C08.mpt',
 'data\\20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_17_GEIS_C08.mpt']

In [432]:
def data_cleaner(filepath):
    ## open text file
    with open(filepath, 'r+') as f:
        file_contents = f.read()
    ## split the text
    text = file_contents.split('\n\n')
    
    if 'GEIS' in filepath:
        labor_type = 'GEIS'
    else:
        if 'GCPL6' in filepath:
            labor_type = 'GCPL6'
    
    os.makedirs('data/'+labor_type, exist_ok=True)
    
    '''setting data'''
    setting = text[2]
    setting = setting.split('Cycle Definition : Charge/Discharge alternance\n')[1]
    ## convert ',' to '.' for decimal
    setting = setting.replace(',','.')
    tmp = []
    for t in setting.split('\n'):
        tmp.append(re.split(r'\s{3,}', t.strip()))
    setting = tmp
    setting = np.transpose(setting)  
    #setting = pd.read_table(setting,header=None,engine='python').T
    with open("data/"+labor_type+"/setting.csv", "w",encoding='utf-8') as text_file:
        writer = csv.writer(text_file)
        writer.writerows(setting)
        
        
    '''loop data'''
    tmp = []
    for t in text[3].split('\n'):
        tmp.append([int(s) for s in str(t).split() if s.isdigit()])
    tmp =tmp[1:]
    loop_data = tmp
    with open("data/"+labor_type+"/loop_data.csv", "w",encoding='utf-8') as text_file:
        writer = csv.writer(text_file)
        writer.writerows(loop_data)
    
    
    '''labor data'''
    ## convert ',' to '.' for decimal
    labor_data = text[-1].replace(',','.')
    with open("data/"+labor_type+"/labor_data.csv", "w",encoding='utf-8') as text_file:
        text_file.write(labor_data)
        
        
        

In [433]:
for path in datafilespath:
    data_cleaner(path)